In [42]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta


import xarray as xr
import matplotlib.pyplot as plt
import sys
# sys.path.append(r'C:\Users\ruro\OneDrive - Boskalis\Documents\python\OSSI')
from scipy.signal import welch
sys.path.append(r'C:\Users\dpoppema\Documents\GitHub\HybridDune\Ruben\Pressure_sensors\S1\RBR_05')
import puv 
#from cdo import *

In [43]:
# # LOAD RAW DATA
# Physical constants -----------------------------------------------------------------------------------
rho = 1027 #kg/m3, for seawater at 9C, avg temp at HKZ measurement station
g = 9.8125  # value Zandmotor

# # input parameters per file ----------------------------------
subfolder_in_all = ['refP1 RBR4','S1P2 RBR3', 'S1P3 RBR5', 'S2P3 RBR1', 'S3P3 RBR6', 'S4P3 RBR2'] # subfolder where file is sitting within experimentFolder (on O drive Daan)
instrumentname_all = subfolder_in_all #['refP1 RBR4', 'S3P6 RBR6']
sf_all = [8, 8, 16, 8, 16, 8] #[hz] sampling frequency

# # input parameters general ---------------------------------------------
experimentFolder = r'O:\HybridDune experiment\data RBR, OSSI\copy RBR Udrive series1'                  # path where the data is sitting # Rubens Laptop


In [ ]:
dfp.P.plot()
start_time = pd.Timestamp("2024-12-13T16:00:00")
end_time = pd.Timestamp("2024-12-27T13:00:00")
plt.xlim(start_time, end_time)
pt
# plt.xlim(ds0.t.min(), ds0.t.max())

In [ ]:
# ds0.p.plot()
ds0.pc.plot()
plt.grid()


In [ ]:
plt.figure(figsize=(16,2))
ds0.pc.plot()
plt.grid()

start_time = pd.Timestamp("2024-12-17T16:00:00")
end_time = pd.Timestamp("2024-12-23T13:00:00")
plt.xlim(start_time, end_time)

In [ ]:
plt.figure(figsize=(16,2))
ds0.pc.plot()
plt.grid()

start_time = pd.Timestamp("2024-12-17T20:00:00")
end_time = pd.Timestamp("2024-12-18T13:00:00")
plt.xlim(start_time, end_time)

In [ ]:
plt.figure(figsize=(16,2))
ds0.pc.plot()
plt.grid()

start_time = pd.Timestamp("2024-12-17T20:00:00")
end_time = pd.Timestamp("2024-12-17T20:05:00")
plt.xlim(start_time, end_time)
plt.ylim(-100, 20000)

In [44]:
i = 4
# frequency resolution in fourier space
### delta_f = p_blocks/D_timeframe ###
p_blocks = 30   # number of segments within block, for the Welch method
D_length = 1800 # Duration of block in seconds (30 minutes)
fresolution = p_blocks / D_length # Frequency resolution is 1/T_segment = n_segments / T_block
nperseg = D_length * sf_all[i] / p_blocks - 0.5 #dim should be len(ds.f); whelch has (nperseg/2 +1)
instrument = instrumentname_all[i]  # select instrument

instrFile = os.path.join(experimentFolder,'QC', instrument +' p_rel.nc')
ncOutFile = os.path.join(experimentFolder,'tailored', instrument +'.nc')


# %% load the raw data from netcdf
ds0 = xr.open_dataset(instrFile)
print(np.size(ds0.p.values), 'time steps in dataset')

#let's remove the bursts where there are only nans
#ds0 = ds0.dropna(dim='t')                                      # temp skipped

# make a new dataset that has an extra dimension to accomodate for the frequency axis
ds = xr.Dataset(data_vars={},
                coords={'t': ds0.t.values,
                        'N': ds0.N.values,
                        'f': np.arange(0, ds0.sf.values / 2, fresolution)})
ds['f'].attrs = {'units': 'Hz'}
ds.attrs = ds0.attrs

# put all variables in this new dataset
for key in ds0.data_vars:
    ds[key] = ds0[key]


: 

In [31]:
print(np.size(ds.N))


19200


In [40]:
# temp: resize p -----------------------------------------------------

# Create a continuous time array
# Make full time vector
t0 = datetime(2024, 12, 12, 9, 0, 0)
t_end = datetime(2024, 12, 27, 16, 20, 0)
t_full   = pd.date_range(t0, t_end, freq='{}s'.format(1 / 16)) # 16hz time vector
x = np.arange(0, len(t_full))  # create an index for the full time vector;

ds_new = xr.Dataset(data_vars={},
                coords={'t_full': t_full,
                        't': ds0.t.values,
                        'N': ds0.N.values,
                        'f': np.arange(0, ds0.sf.values / 2, fresolution)})

# put all variables in this new dataset
for key in ds.data_vars:
    ds_new[key] = ds[key]

print(ds_new)
print(np.size(ds_new.p.values), 'size p')

# Save with compression
ncFilePath1 = r'O:\HybridDune experiment\data RBR, OSSI\copy RBR Udrive series1\QC\copy.nc'
ncFilePath2 = r'O:\HybridDune experiment\data RBR, OSSI\copy RBR Udrive series1\QC\copy with t_full.nc'
ncFilePath3 = r'O:\HybridDune experiment\data RBR, OSSI\copy RBR Udrive series1\QC\copy deflate4.nc'
ncFilePath4 = r'O:\HybridDune experiment\data RBR, OSSI\copy RBR Udrive series1\QC\copy with t_full deflate4.nc'

encoding3 = {var: {"zlib": True, "complevel": 4} for var in ds.data_vars}
encoding4 = {var: {"zlib": True, "complevel": 4} for var in ds_new.data_vars}


#ds.to_netcdf(    ncFilePath1)
#ds_new.to_netcdf(ncFilePath2)
#ds.to_netcdf(    ncFilePath3, encoding=encoding3)
#ds_new.to_netcdf(ncFilePath4, encoding=encoding4)
nBursts = 256
burstLength = 19200  # number of samples per burst
#ds_new['t_2d'] = (('t', 'N'), t_full.reshape((nBursts, burstLength)))      # relative pressure, pAir subtracted
ds_new['t_2d'] = (('t', 'N'), x.reshape((nBursts, burstLength)))      # relative pressure, pAir subtracted

# reshape 
#ds_new.p2 = ds_new.t_2d.stack(z=('t_full', 'N')).reset_index('z')
#ds_new.p2.attrs = ds_new.p.attrs
#ds_new.p2.name = 'p2'

# save reshaped
ncFilePath5 = r'O:\HybridDune experiment\data RBR, OSSI\copy RBR Udrive series1\QC\copy with t_full reshape.nc'
ncFilePath6 = r'O:\HybridDune experiment\data RBR, OSSI\copy RBR Udrive series1\QC\copy with t_full reshape deflate4.nc'
encoding6 = {var: {"zlib": True, "complevel": 4} for var in ds_new.data_vars}
#ds_new.to_netcdf(ncFilePath5)
#ds_new.to_netcdf(ncFilePath6, encoding=encoding6)


<xarray.Dataset> Size: 248MB
Dimensions:  (t_full: 21158401, t: 256, N: 19200, f: 480)
Coordinates:
  * t_full   (t_full) datetime64[ns] 169MB 2024-12-12T09:00:00 ... 2024-12-27...
  * t        (t) datetime64[ns] 2kB 2024-12-17T14:20:00 ... 2024-12-23T10:40:00
  * N        (N) float64 154kB 0.0 0.0625 0.125 ... 1.2e+03 1.2e+03 1.2e+03
  * f        (f) float64 4kB 0.0 0.01667 0.03333 0.05 ... 7.933 7.95 7.967 7.983
Data variables:
    p        (t, N) float64 39MB 6.616e+03 6.548e+03 ... 1.357e+04 1.357e+04
    zi       float64 8B ...
    zb       float64 8B ...
    sf       int64 8B 16
    p_rel    (t, N) float64 39MB 6.616e+03 6.548e+03 ... 1.357e+04 1.357e+04
    name     <U9 36B ...
4915200 size p


ValueError: cannot reshape array of size 21158401 into shape (256,19200)

In [39]:
print(np.size(ds.p)/19200)
nperseg = D_length * sf_all[i] / p_blocks - 0.5 #dim should be len(ds.f); whelch has (nperseg/2 +1)
a = np.arange(0,5)
print(len(t_full))

256.0
21158401


In [ ]:
# # DAAN: REMOVE FILTERING WITH OWN VERSION !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

def remove_outliers(data, threshold=3):
    mean = np.mean(data)
    std_dev = np.std(data)
    mask = np.abs(data - mean) < threshold * std_dev
    # filtered_data = np.where(mask, data, mean + threshold * std_dev * np.sign(data - mean))  # Replace outliers with closest value within threshold
    filtered_data = np.where(mask, data, np.nan) # Replace outliers with nan
    return filtered_data

# Create a lambda function to pass the threshold parameter
ufunc = lambda x: remove_outliers(x, threshold=3)

# Apply the outlier removal function to the pressure data
ds['p'] = xr.apply_ufunc(ufunc, 
                          ds['p'],
                          input_core_dims=[['N']],
                          output_core_dims=[['N']],
                          vectorize=True)

# Interpolate NaN values in the pressure signal
ds['p'] = ds['p'].interpolate_na(dim='N', method='linear')


ds.p.isel(t=111).plot()

In [ ]:
# Print the initial number of NaN values
initial_nan_count = np.sum(np.isnan(ds['p'].values))
print("Initial NaN count:", initial_nan_count)

# Interpolating NaN values using linear interpolation
ds['p'] = ds['p'].interpolate_na(dim='N', method='linear')

# Handling any remaining NaN values
ds['p'] = ds['p'].ffill(dim='N').bfill(dim='N')

# Print the number of NaN values after interpolation
remaining_nan_count = np.sum(np.isnan(ds['p'].values))
print("Remaining NaN count:", remaining_nan_count)

In [ ]:
### pressure filtering ####
ufunc = lambda x: puv.band_pass_filter2(ds.sf.values, x, fmin=0.004, fmax=1.5)

ds['p'] = xr.apply_ufunc(ufunc, 
                          ds['p'],
                          input_core_dims=[['N']],
                          output_core_dims=[['N']],
                          vectorize=True)

# Compute water depth
ds['h'] = (ds['p'] / (rho * g) + (ds['zi'] - ds['zb'])).mean(dim='N')  # Relative to bed level
ds['h'].attrs = {'long_name': 'mean water level', 'units': '[m] above bed level'}



In [ ]:
print( np.size(ds.p.values) /19200 )
print( np.size(ds.h.values) )


In [ ]:
# Several wave statistics computations, only based on pressure, for full bandpass
Threshold= 4    # DAAN: SKIP


ufunc = lambda x, h: puv.attenuate_signal(
    'pressure', 
    ds.sf.values, x, h, 
    ds.zi.values,
    ds.zb.values,
    rho=1027,
    g=9.8125,
    removeNoise=False,
    detrend=True)

fx, ds['zs'] = xr.apply_ufunc(ufunc, 
                              ds['p'], ds['h'],
                              input_core_dims=[['N'], []],
                              output_core_dims=[['f'], ['N']],
                              vectorize=True)

ufunc = lambda zs: remove_outliers(zs, threshold=Threshold)   # DAAN: SKIP

# ds['zs'] = xr.apply_ufunc(ufunc, 
#                           ds['zs'],
#                           input_core_dims=[['N']],
#                           output_core_dims=[['N']],
#                           vectorize=True)

# # Interpolate NaN values in the pressure signal
# ds['zs'] = ds['zs'].interpolate_na(dim='N', method='linear')

# Drop values that are below bed level. 
# ds['zs'] = ds['zs'].where(ds['zs'] >= 0.0, drop=True)

ds['zs'].attrs = {'units': 'm', 'long_name': 'surface elevation'}   # DAAN: CHECK: MORE METADATA NEEDED?

ufunc = lambda p: welch(p, fs=ds.sf.values, nperseg=nperseg, detrend='constant', window='hann') # Detrend: is per segment. 1min, about constant, so false used

ds['frequencies'], ds['psd'] = xr.apply_ufunc(ufunc,
                                                ds['zs'],
                                                input_core_dims=[['N']],
                                                output_core_dims=[['f'], ['f']],
                                                vectorize=True)

ufunc = lambda psd: puv.compute_wave_params(ds.f.values, psd, fmin=0.004 , fmax=5)   # DAAN: CHECK: WHY fmax=5? Different result than 1.5?

ds['Hm0'], ds['Tp'], ds['Tm01'], ds['Tm02'], ds['Tmm10'], ds['Tps'] = xr.apply_ufunc(ufunc,
                                                                          ds['psd'],
                                                                          input_core_dims=[['f']],
                                                                          output_core_dims=[[], [], [], [], [], []],
                                                                          vectorize=True)



In [ ]:
## low frequencies: IG waves ##
## now first filter bandpass, then compute wave_params

ufunc = lambda x: puv.band_pass_filter2(ds.sf.values, x, fmin=0.004, fmax=0.05)

ds['p_low'] = xr.apply_ufunc(ufunc, 
                          ds['p'],
                          input_core_dims=[['N']],
                          output_core_dims=[['N']],
                          vectorize=True)

ufunc = lambda x, h: puv.attenuate_signal(
    'pressure', 
    ds.sf.values, x, h, 
    ds.zi.values,
    ds.zb.values,
    rho=1027,
    g=9.8125,
    removeNoise=False,
    detrend=True)

fx, ds['zs_low'] = xr.apply_ufunc(ufunc, 
                              ds['p_low'], ds['h'],
                              input_core_dims=[['N'], []],
                              output_core_dims=[['f'], ['N']],
                              vectorize=True)

# ufunc = lambda zs: remove_outliers(zs, threshold=Threshold)

# ds['zs_low'] = xr.apply_ufunc(ufunc, 
#                           ds['zs_low'],
#                           input_core_dims=[['N']],
#                           output_core_dims=[['N']],
#                           vectorize=True)

# ds['zs_low'] = ds['zs_low'].interpolate_na(dim='N', method='linear')

# ds['zs_low'] = ds['zs_low'].where(ds['zs_low'] >= 0.0, drop=True)

ds['zs_low'].attrs = {'units': 'm', 'long_name': 'surface elevation low freq.'}


ufunc = lambda p: welch(p, fs=ds.sf.values, nperseg=nperseg, detrend='constant', window='hann')

ds['frequencies_low'], ds['psd_low'] = xr.apply_ufunc(ufunc,
                                                ds['zs_low'],
                                                input_core_dims=[['N']],
                                                output_core_dims=[['f'], ['f']],
                                                vectorize=True)

ufunc = lambda psd: puv.compute_wave_params(ds.f.values, psd, fmin=0.004 , fmax=0.05)

ds['Hm0_low'], ds['Tp_low'], ds['Tm01_low'], ds['Tm02_low'], ds['Tmm10_low'], ds['Tps_low'] = xr.apply_ufunc(ufunc,
                                                                          ds['psd_low'],
                                                                          input_core_dims=[['f']],
                                                                          output_core_dims=[[], [], [], [], [], []],
                                                                          vectorize=True)




In [ ]:
## High frequencies: wind-waves ##
## now first filter bandpass, then compute wave_params

ufunc = lambda x: puv.band_pass_filter2(ds.sf.values, x, fmin=0.05, fmax=5)

ds['p_high'] = xr.apply_ufunc(ufunc, 
                          ds['p'],
                          input_core_dims=[['N']],
                          output_core_dims=[['N']],
                          vectorize=True)

ufunc = lambda x, h: puv.attenuate_signal(
    'pressure', 
    ds.sf.values, x, h, 
    ds.zi.values,
    ds.zb.values,
    rho=1027,
    g=9.8125,
    removeNoise=False,
    detrend=True)

t, ds['zs_high'] = xr.apply_ufunc(ufunc, 
                              ds['p_high'], ds['h'],
                              input_core_dims=[['N'], []],
                              output_core_dims=[['f'], ['N']],
                              vectorize=True)

ufunc = lambda zs: remove_outliers(zs, threshold=Threshold)

# ds['zs_high'] = xr.apply_ufunc(ufunc, 
#                           ds['zs_high'],
#                           input_core_dims=[['N']],
#                           output_core_dims=[['N']],
#                           vectorize=True)

# ds['zs_high'] = ds['zs_high'].interpolate_na(dim='N', method='linear')

# ds['zs_high'] = ds['zs_high'].where(ds['zs_high'] >= 0.0, drop=True)

ds['zs_high'].attrs = {'units': 'm', 'long_name': 'surface elevation high freq.'}



ufunc = lambda p: welch(p, fs=ds.sf.values, nperseg=nperseg, detrend='constant', window='hann')

ds['frequencies_high'], ds['psd_high'] = xr.apply_ufunc(ufunc,
                                                ds['zs_high'],
                                                input_core_dims=[['N']],
                                                output_core_dims=[['f'], ['f']],
                                                vectorize=True)

ufunc = lambda psd: puv.compute_wave_params(ds.f.values, psd, fmin=0.05 , fmax=5)

ds['Hm0_high'], ds['Tp_high'], ds['Tm01_high'], ds['Tm02_high'], ds['Tmm10_high'], ds['Tps_high'] = xr.apply_ufunc(ufunc,
                                                                          ds['psd_high'],
                                                                          input_core_dims=[['f']],
                                                                          output_core_dims=[[], [], [], [], [], []],
                                                                          vectorize=True)


In [ ]:
## skewness of waves ##
ufunc = lambda p: puv.compute_SkAs(ds.sf.values,p,fpfac =None, fbounds = None)   

ds['Sk'], ds['As'], ds['sig'] =  xr.apply_ufunc(ufunc,
                                                ds['p'], 
                                                input_core_dims=[['N']],
                                                output_core_dims=[[], [], []],
                                                vectorize=True)


In [ ]:
# Flatten all variables with dimension 'N'
# for var in ds.data_vars:
#     if 'N' in ds[var].dims:
#         ds[var] = ds[var].stack(t_N=('t', 'N'))

# %% write to file
# we strip all information on burst scale from the dataset to reduce size (and this info is already present in the raw_netcdf version of the data)
# dsTailored = ds.drop_dims('N')
dsTailored = ds


if not os.path.isdir(os.path.join(experimentFolder,'tailored')):
    os.mkdir(os.path.join(experimentFolder,'tailored'))
ncFilePath = os.path.join(experimentFolder, 'tailored', ds0.instrument + '.nc')
dsTailored.to_netcdf(ncFilePath)



In [ ]:
# Save with compression
ncFilePath = os.path.join(experimentFolder, 'tailored', ds0.instrument + 'deflate4.nc')
encoding = {var: {"zlib": True, "complevel": 6} for var in dsTailored.data_vars}
dsTailored.to_netcdf(ncFilePath, encoding=encoding)

In [ ]:
# Reshape XArray ds

# Make full time vector
t0 = datetime(2024, 12, 12, 9, 0, 0)
t_end = datetime(2024, 12, 27, 16, 20, 0)
t_full   = pd.date_range(t0, t_end, freq='{}s'.format(1 / 16)) # 16hz time vector

# Add t_full to dsTailered as additional dimension
dsTailored = dsTailored.assign_coords(t_full=t_full)
dsTailored = dsTailored.expand_dims('t_full')

In [ ]:
# Create a continuous time array
t_continuous = np.array([t + np.timedelta64(int(n), 's') for t in ds.t.values for n in ds.N.values])

# Flatten the burst structure for plotting
zs_flat = ds.zs.values.flatten()
zs_low_flat = ds.zs_low.values.flatten()
zs_high_flat = ds.zs_high.values.flatten()


In [ ]:
# Plotting
plt.figure(figsize=(15, 8))

plt.plot(t_continuous, zs_flat, label='zs', alpha=0.7, linewidth=0.9)
plt.plot(t_continuous, zs_high_flat, label='zs_high', alpha=0.7, linewidth=0.9)
plt.plot(t_continuous, zs_low_flat, label='zs_low', alpha=0.7, linewidth=0.9)

ds.h.plot(label='Waterlevel')



# give values below y=0 a different color
# plt.fill_between(t_continuous, zs_flat, 0, where=zs_flat < 0, color='grey', alpha=1, zorder=4, linewidth=1.5)
# plt.fill_between(t_continuous, zs_low_flat, 0, where=zs_low_flat < 0, color='grey', alpha=1, zorder=4, linewidth=1.5)
# plt.fill_between(t_continuous, zs_high_flat, 0, where=zs_high_flat < 0, color='grey', alpha=1, zorder=4, linewidth=1.5)
plt.axhline(y=0, color='k', linestyle='--', label='Bed level')

# Set x-axis limits using specific date stamps
start_time = pd.Timestamp("2024-12-18T12:00:00")
end_time = pd.Timestamp("2024-12-21T00:00:00")
# plt.xlim(start_time, end_time)

plt.axhline((ds.zi-ds.zb).values, color='k', linestyle='--', label='Instrument height above bed')

plt.xlabel('Time')
plt.ylabel('Surface Elevation (m)')
plt.title('Surface Elevation vs Time RBR06 2024-12-16 to 2024-12-27')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

In [ ]:
# plot hmo 
plt.figure(figsize=(15, 8))
ds.Hm0.plot(label='Hm0')
ds.Hm0_low.plot(label=f'Hm0_low (mean={ds.Hm0_low.mean().values:.3g} m)')

ds.Hm0_high.plot(label='Hm0_high')
plt.legend()

plt.xlabel('Time')
plt.ylabel('Hm0 (m)')
plt.title('Hm0 vs Time')
plt.grid(True)

# Set x-axis limits using specific date stamps
start_time = pd.Timestamp("2024-12-18T12:00:00")
end_time = pd.Timestamp("2024-12-21T00:00:00")
# plt.xlim(start_time, end_time)

plt.show()


In [ ]:
# plot hs and h
plt.figure(figsize=(16, 3))
ds.h.plot(label='Waterlevel')
# (ds.h + ds.zb).plot(label='Waterlevel + zb (NAP)')

# Interpolate between points
ds.Hm0.plot(label=f'Hm0 (mean={ds.Hm0.mean().values:.3g} m)')
ds.Hm0_low.plot(label=f'Hm0_low (mean={ds.Hm0_low.mean().values:.3g} m)')

# ds.Hm0_high.plot(label='Hm0_high')
plt.legend()

plt.xlabel('Time')
plt.ylabel('(m)')
plt.title('Hm0 vs Time')
plt.grid(True)

# Set x-axis limits using specific date stamps
start_time = pd.Timestamp("2024-12-21T16:00:00")
end_time = pd.Timestamp("2024-12-23T13:00:00")
# plt.xlim(start_time, end_time)

plt.show()

In [ ]:
t = 111
end =120

plt.figure(figsize=(16, 2))
(ds.zs.isel(t=t)-ds.zs.isel(t=t).mean()).plot(label = f'Water elevation; Hm0: {ds.Hm0.isel(t=t).values:.3g}m')
# (ds.zs.isel(t=t)-ds.h.isel(t=t).mean()).plot(label = f'Water elevation; Hm0: {ds.Hm0.isel(t=t).values:.3g}m')
plt.grid()
plt.xlim(0,end)
plt.ylabel(f'\u03B7 [m]')
plt.legend(loc='upper right')

plt.figure(figsize=(16, 2))
ds.zs.isel(t=t).plot(color='grey', label= 'total elevation')
ds.zs_low.isel(t=t).plot(color='green', label = f'infragravity waves; Hm0: {ds.Hm0_low.isel(t=t).values:.3g}m')
plt.grid()
plt.xlim(0,end)
plt.legend(loc='upper right')

plt.figure(figsize=(16, 2))
ds.zs.isel(t=t).plot(color='grey', label= 'total elevation')
ds.zs_high.isel(t=t).plot(color='red', label = f'wind waves; Hm0: {ds.Hm0_high.isel(t=t).values:.3g}m')

plt.grid()
plt.xlim(0,end)
plt.legend(loc='upper right');

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to remove outliers
# def remove_outliers(data, threshold=3):
#     mean = np.mean(data)
#     std_dev = np.std(data)
#     filtered_data = [x if abs(x - mean) < threshold * std_dev else mean for x in data]
#     return np.array(filtered_data)

def remove_outliers(data, threshold=3):
    mean = np.mean(data)
    std_dev = np.std(data)
    mask = np.abs(data - mean) < threshold * std_dev
    # filtered_data = np.where(mask, data, mean + threshold * std_dev * np.sign(data - mean))  # Replace outliers with closest value within threshold
    filtered_data = np.where(mask, data, np.nan) # Replace outliers with nan
    return filtered_data


data = ds.zs.isel(t=t).values  # Extract the data values for the selected time index t

# Remove outliers from the data
cleaned_data = remove_outliers(data)

# Interpolate NaN values in the pressure signal
# cleaned_data = cleaned_data.interpolate_na(method='linear')

# Calculate the mean of the cleaned data
mean_cleaned_data = np.mean(cleaned_data)

# Plotting the cleaned data
plt.figure(figsize=(16, 2))
plt.plot(ds.N, cleaned_data - mean_cleaned_data, label=f'Water elevation; Hm0: {ds.Hm0.isel(t=t).values:.3g}m')
plt.grid()
plt.xlim(0, 200)
plt.ylabel(f'\u03B7 [m]')
plt.legend(loc='upper right')
plt.show()

print(len(cleaned_data)/16)

In [ ]:
from scipy.stats import chi2
t=111
freq = ds.frequencies.isel(t=t)
psd = ds.psd.isel(t=t)


# Calculate confidence intervals
nBlocks = len(ds.zs.isel(t=t)) // nperseg  # Estimate the number of blocks used by Welch's method
edf = round(nBlocks * 2)     # Degrees of freedom (approximately 2 per segment)
alpha = 0.1                  # 90% confidence level

confLow = edf / chi2.ppf(1 - alpha / 2, edf)  # Lower confidence limit
confUpper = edf / chi2.ppf(alpha / 2, edf)    # Upper confidence limit

# Confidence interval bounds for PSD
psd_lower = psd * confLow
psd_upper = psd * confUpper



### background plot 
freq_bg, psd_bg = welch(ds.zs.isel(t=t).values, fs=16, nperseg=19200, detrend='constant', window='hann')
nBlocks_bg = len(ds.zs.isel(t=t).values) // 19200  # Estimate the number of blocks used by Welch's method
edf_bg = round(nBlocks * 2)     # Degrees of freedom (approximately 2 per segment)
confLow_bg = edf / chi2.ppf(1 - alpha / 2, edf)  # Lower confidence limit
confUpper_bg = edf / chi2.ppf(alpha / 2, edf)    # Upper confidence limit
psd_lower_bg = psd_bg * confLow_bg
psd_upper_bg = psd_bg * confUpper_bg
# plt.plot(freq_bg, psd_upper_bg, color='gray', linestyle= '--')
plt.fill_between(freq_bg, psd_lower_bg, psd_upper_bg, color='gray', alpha=0.3, label='Raw spectrum', linestyle='--')

### plotting
plt.fill_between(freq, psd_lower, psd_upper, color='black', alpha=0.3, label='90% Confidence Interval')
plt.semilogy(freq, psd, label='Power Density Spectrum')

plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD (m²/Hz)')
plt.title(f'Power Spectral Density using Welch\'s Method for t ={t}')
plt.grid()
plt.yscale('linear')
# plt.xscale('log')
plt.xscale('linear')
plt.xlim(0,0.6)
plt.legend()
plt.show()

In [ ]:
def find_time_index(ds, specific_time):
    """
    Function to find the index 't' for a specific time in the dataset.
    
    Parameters:
    ds (xarray.Dataset): The dataset containing the time dimension.
    specific_time (str or datetime): The specific time to find the index for.
    
    Returns:
    int: The index corresponding to the specific time.
    """
    specific_time = np.datetime64(specific_time)
    time_index = np.where(ds.t.values == specific_time)[0]
    
    if len(time_index) == 0:
        raise ValueError(f"Time {specific_time} not found in the dataset.")
    
    return time_index[0]

# Example usage
specific_time = "2024-12-20T12:00:00"
t_index = find_time_index(ds, specific_time)
print(f"The index for the specific time {specific_time} is {t_index}.")